# Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# (GPU) Install autogluon

!pip3 install -U pip
!pip3 install -U setuptools wheel

!pip3 install autogluon

     |████████████████████████████████| 2.1 MB 5.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.0/267.0 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# # (CPU) Install autogluon (https://auto.gluon.ai/stable/index.html#installation)

# !pip3 install -U pip
# !pip3 install -U setuptools wheel

# # CPU version of pytorch has smaller footprint - see installation instructions in
# # pytorch documentation - https://pytorch.org/get-started/locally/
# !pip3 install torch==1.10.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

# !pip3 install autogluon

In [ ]:
import matplotlib as plt
%matplotlib inline
import autogluon.core as ag
from autogluon.vision import ImagePredictor, ImageDataset
import pandas as pd
import numpy as np
import os

# Make inference

In [ ]:
MY_PATH = "drive/MyDrive/COMS E6998 PDL Sys Perf/Project/"
os.chdir(MY_PATH)

In [ ]:
os.getcwd()

'/content/drive/MyDrive/COMS E6998 PDL Sys Perf/Project'

In [ ]:
# test_image = "dl-project/test/destroyed/santa-rosa-wildfire_00000204_post_disaster.png"
test_image = "santa-rosa-wildfire_00000204_post_disaster.png"

In [ ]:
# Load predictor
filename = 'autogluon-predictor.ag'
# predictor.save(filename)
predictor = ImagePredictor.load(filename)

In [ ]:
label_dict = {'no-damage': 1, 'destroyed': 0}

In [ ]:
result = predictor.predict(test_image)
print(result)

0    0
Name: label, dtype: int64


# Appendix: Load data

In [ ]:
MY_PATH = "drive/MyDrive/COMS E6998 PDL Sys Perf/Project/"
os.chdir(MY_PATH)

In [ ]:
train_data, val_data, test_data = ImageDataset.from_folders('dl-project', train='train', val='validation', test='test')
print('train #', len(train_data), 'test #', len(test_data))
train_data.head()

# label: 0 = "destroyed", 1 = "no-damage"

train # 1727 test # 583


,image,label
0,/content/drive/MyDrive/COMS E6998 PDL Sys Perf...,0
1,/content/drive/MyDrive/COMS E6998 PDL Sys Perf...,0
2,/content/drive/MyDrive/COMS E6998 PDL Sys Perf...,0
3,/content/drive/MyDrive/COMS E6998 PDL Sys Perf...,0
4,/content/drive/MyDrive/COMS E6998 PDL Sys Perf...,0


In [ ]:
train_data['label'].value_counts()

1    1370
0     357
Name: label, dtype: int64

In [ ]:
val_data['label'].value_counts()

1    437
0    125
Name: label, dtype: int64

In [ ]:
test_data['label'].value_counts()

1    476
0    107
Name: label, dtype: int64

In [ ]:
test_data.show_images()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
filename = 'predictor.ag'
# predictor.save(filename)
predictor = ImagePredictor.load(filename)

In [ ]:
fit_result = predictor.fit_summary()
print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Top-1 train acc: 0.863, val acc: 0.893


In [ ]:
bulk_result = predictor.predict(test_data)
print(bulk_result)

0      1
1      1
2      1
3      1
4      1
      ..
578    0
579    0
580    0
581    1
582    1
Name: label, Length: 583, dtype: int64


In [ ]:
test_acc = predictor.evaluate(test_data)
print('Top-1 test acc: %.3f' % test_acc['top1'])

[Epoch 7] validation: top1=0.883362 top5=1.000000


Top-1 test acc: 0.883


In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
M = confusion_matrix(np.array(test_data['label']), np.array(bulk_result))
tn, fp, fn, tp = M.ravel() 

In [ ]:
# 449 27
#  41 66

# 94%  6% (no damage)
# 38% 62%  (destroyed)

In [ ]:
# use predictor_loaded as usual
result = predictor_loaded.predict(image_path)
print(result)

# Appendix: Train predictor

In [ ]:
predictor = ImagePredictor()

In [ ]:
# since the original dataset does not provide validation split, the `fit` function splits it randomly with 90/10 ratio
predictor.fit(train_data, tuning_data=val_data)  # you can trust the default config, we reduce the # epoch to save some build time


`time_limit=auto` set to `time_limit=7200`.
Reset labels to [0, 1]
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Reset labels to [0, 1]
The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1
Starting fit without HPO
modified configs(<old> != <new>): {
root.train.epochs    200 != 15
root.train.early_stop_baseline 0.0 != -inf
root.train.batch_size 32 != 16
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.img_cls.model   resnet101 != resnet50
root.misc.num_workers 4 != 2
root.misc.seed       42 != 432
}
Saved config to /content/drive/MyDrive/COMS E6998 PDL Sys Perf/Project/c14df13e/.trial_0/config.yaml
Model resnet50 created, param count:                                         23512130
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 7.767785 samples/sec	accuracy=0.701250	lr=0.000100
Epoch[0] Batch [99]	S

# Appendix: Model choices

In [ ]:
ImagePredictor.list_models()

('adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x